# Stationary Descendents and the Discriminant Modular Form

This Jupyter notebook provides all of the classes and functions used for the calculations in the paper 'Stationary Descendents and the Discriminant Modular Form'.

At the heart of this notebook are two classes:

1. Stationary_Descendent()
2. Descendent_Matroid()

The Stationary_Descendent class allows one to compute the **disconnected stationary descendent Gromov-Witten invariants**,

\begin{equation*}
\left<\tau_{k_1}\ldots\tau_{k_n}\right>^{\bullet E}_d
\end{equation*}

along with (truncations of) the **stationary descendents of weight $k = \sum(k_i + 2)$**

\begin{equation*}
\left<\tau_{k_1}\ldots\tau_{k_n}\right> := \prod_{n \geq 1}(1 - q^n)\sum_{d \geq 0}\left<\tau_{k_1}\ldots\tau_{k_n}\right>^{\bullet E}_d q^d
\end{equation*}

The Descendent_Matroid class allows one to make calculations with the **descendent matroid of weight $k$** i.e. the matroid whose ground set is the set of stationary descendents of weight $k$, and whose bases are collections of stationary descendents that form a basis of the vector space of **quasimodular forms of weight $k$**.

In [1]:
from itertools import *
R.<q> = QQ[]

The Gromov-Witten invariants $\left<\tau_{k_1}\ldots\tau_{k_n}\right>^{\bullet E}_d$ have the following closed formula,

\begin{equation}
\left<\tau_{k_1}\tau_{k_2}\ldots\tau_{k_n}\right>^{\bullet E}_d = \left(\prod_{i = 1}^n\frac{1}{(k_i + 1)!}\right)\sum_{\lambda, \mu \ \vdash \ d}\frac{1}{z_\mu}\left(\chi^\lambda_\mu\right)^2\prod_{i = 1}^np_{k_i + 1}(\lambda)
\end{equation}

The first goal is to write a function that computes this Gromov-Witten invariant. This function will require

1. The centralizer order $z_\mu$
2. The irreducible character values $\chi^\lambda_\mu$
3. The symmetric power sum values $p_k(\lambda)$

Let $\mu = (\mu_1, \ldots, \mu_k) \vdash d$, and let $m_i$ be the **multiplicity of $i$** in $\mu$ i.e. the number of times the integer $i$ shows up in $\mu$. The centralizer order $z_\mu$ is computed by

\begin{equation*}
z_\mu = \left(\prod_{i = 1}^{|\mu|}m_i!\right)\left(\prod_{i = 1}^k\mu_i\right)
\end{equation*}

We'll write a function called centralizer(), that computes $z_\mu$.

In [2]:
def centralizer(mu):
    ### Takes a partition $\mu$ of $d$ and returns  ###
    ### the order of the centralizer corresponding  ###
    ### to $\mu$.
    
    if mu == []: # Returns 1 if $\mu$ is the empty partition
        return 1
    
    m = max(mu) # Don't need to check for multiplicities past the max entry of \mu 
    
    multiplicity_counter = [] # Instantiate the list of multiplicities
    
    for i in range(1, m + 1): # Creates the list [m_1, m_2, ...]
        multiplicity_of_i = 0
        for j in mu:
            if j == i:
                multiplicity_of_i += 1
        multiplicity_counter.append(multiplicity_of_i)
        
    first_factor = 1
    
    for i in multiplicity_counter: # Creates the product of factorials of multiplicities
        first_factor *= factorial(i)
    
    second_factor = 1
    
    for i in mu: # Creates the product of parts of \mu 
        second_factor *= i
        
    return first_factor*second_factor

Next, we need a function to compute the irreducible character values $\chi^\lambda_\mu$. Lizzie Hernandez (https://github.com/lizziehv) has written a fast implentation of a recursive version of the Murnaghan-Nakayama rule, which we provide below. She uses the dynamical programming technique of 'memoization', which speeds up the recursive procedure tremendously.

In [3]:
def reduce(shape, s):
    th_list = [] # list of (𝜆∖𝑟, ℎ(𝑟)) pairs
    
    border_size = [] # list containing number of border cells in every row
    for r in range(len(shape)):
        border_size.append(shape[r] if r == len(shape) - 1
                           else shape[r] - shape[r + 1] + 1)
    
    for r_start_idx in range(len(shape)):
        new_shape = shape.copy()
        
        removed = 0 # number of removed border cells
        r_idx = r_start_idx # index of row from which to remove cells
        while removed < s and r_idx < len(shape):
            rrem = min(border_size[r_idx], s - removed) # number of cells to remove from row at rem_idx
            new_shape[r_idx] -= rrem # remove cells from shape
            
            removed += rrem
            r_idx += 1
        
        # check that shape is valid and that exactly s cells have been removed
        if removed != s or (r_idx < len(shape) and new_shape[r_idx - 1] < new_shape[r_idx]):
            continue
        else:
            # remove 0s from new shape
            new_shape = list(filter(lambda x: x != 0, new_shape))
            th_list.append((new_shape, r_idx - r_start_idx - 1))
    
    return th_list

memo = {}

def character(lda, mu, idx=0, print_flag=False):
    key = (tuple(lda), tuple(mu)) # to index into memo
    
    if key in memo: # return from memo if already computed
        if print_flag:
            print("\t\t"*(idx + 1) + "returns ", memo[key], "from memo")
        return memo[key]
    
    if idx == 0 and print_flag:
        print_tableau(lda)
    if len(lda) == 0 and idx == len(mu):
        if print_flag:
            print("\t\t"*(idx + 1) + "⊥")
            print("\t\t"*(idx + 1) + "returns 1")
        return 1
    
    th_list = reduce(lda, mu[idx])
    char = 0
    
    for t, h in th_list:
        if print_flag:
            print_red_tableau(lda, t, tab="\t\t"*(idx + 1))
        
        ind = -1 if h % 2 == 1 else 1
        char_t = character(t, mu, idx + 1, print_flag)
        char += ind * char_t
    
        if print_flag:
            print("\t\t"*(idx + 1) + "returns ", char_t)
    
    memo[key] = char
    return char

Finally, we need a way to compute

\begin{equation*}
p_k(\lambda) := \sum_{i = 1}^n \left[ \left(\lambda_i - i + \frac{1}{2}\right)^k - \left(-i + \frac{1}{2}\right)^k \right] + (1 - 2^{-k})(-1)^k\frac{B_{k + 1}}{k + 1}
\end{equation*}

We'll write two functions, one that returns $p_k(\lambda)$, and another that returns the product $\prod_{i = 1}^np_{k_i + 1}(\lambda)$

In [4]:
def shifted_symmetric(lmd, k):
    ### Takes a partition $\lmd$ of $d$, and an integer $k$, ### 
    ### and returns p_k(\lambda)                             ###
    
    if k <= 0:
        return 0
    else:
        bernoulli_term = (1 - 2^(-k))*((-1)^k)*(1/(k + 1))*bernoulli(k + 1)
        s = 0
        for i in range(1, len(lmd) + 1):
            s += (lmd[i - 1] - i + (1/2))^(k) - (-i + (1/2))^(k)
        return s + bernoulli_term
    

def product_shifted_symmetric(k, lmd):
    ### Takes a partition $\lmd$ of $d$, and a list [k_1, \ldots, k_n] ###
    ### of integers, and returns \prod_{i = 1}^n p_{k_i + 1}(\lmd)     ###
    
    p = 1
    
    for i in k:
        p *= shifted_symmetric(lmd, i + 1)
        
    return p

We now have all of the functions to compute 

\begin{equation*}
\left<\tau_{k_1}\ldots\tau_{k_n}\right>^{\bullet E}_d = \left( \frac{1}{\prod_{i = 1}^n(k_i + 1)!} \right) \sum_{\lambda, \mu \vdash d}\frac{1}{z_\mu}\left(\chi^{\lambda}_{\mu}\right)^2\prod_{i = 1}^np_{k_i + 1}(\lambda)
\end{equation*}

In [5]:
def descendent(k, d):
    ### Takes a list $[k_1, \ldots, k_n]$ and a degree $d$, and returns          ###  
    ### the Gromov-Witten invariant <\tau_{k_1}\ldots\tau_{k_n}> ^{\cdot E}_{d}  ###
    
    summation = 0
    
    outside_factor = 1
    
    for i in k:
        outside_factor *= (1/factorial(i + 1)) #Create \prod_{i = 1}^n\frac{1}{(k_i + 1)!}
    
    for i in product(Partitions(d), Partitions(d)): #Sum over through \lambda, \mu \vdash d
        lmd = list(i[0])
        mu = list(i[1])
        summation += (1/centralizer(mu))*((character(lmd, mu))^2)*product_shifted_symmetric(k, lmd)
        
    return outside_factor*summation

## The Stationary_Descendent() Class

The goal is to make a class called Stationary_Descendent(). When the user instantiates this class, they will have to provide a list $[k_1, \ldots k_n]$ corresponding to the descendent insertions. The key methods of this class will be

1. Compute $\left<\tau_{k_1}\ldots\tau_{k_n}\right>^{\bullet E}_d$ for any $d \geq 0$
2. Compute $\left<\tau_{k_1}\ldots\tau_{k_n}\right> = \prod(1 - q^n)\sum \left<\tau_{k_1}\ldots\tau_{k_n}\right>^{\bullet E}_d q^d$ up to any order
3. Find $\left<\tau_{k_1}\ldots\tau_{k_n}\right>$ in terms of the Eisenstein basis.



First, we'll make a function called eisenstein(). It will take a weight $k$, and allow the user to compute 

\begin{equation*}
E_k := -\frac{B_k}{2k} + \sum_{n \geq 1} \sigma_{k - 1}(n)q^n
\end{equation*}

up to a prescribed order $n$. Then we'll make a class called Eisenstein_Basis(), that takes a weight $k$, and allows one to work with the Eisenstein basis for the vector space of quasimodular forms of weight $k$.

In [6]:
def eisenstein(weight, order):
    ### Returns the Eisenstein series of weight $k$ up to order $n$ ###
    
    if order == 0:
        return -bernoulli(weight)/(2*(weight))
    else:
        s = -bernoulli(weight)/(2*weight)
        for n in range(1, order + 1):
            s += sigma(n, weight - 1)*q^n
        return s

def truncate(f, n):
    ### Auxiliary helper function, takes a polynomial f(q) and truncates to order n ###
    s = 0
    for i in range(n + 1):
        s += f[i]*q^i
    return s


class Eisenstein_Basis:
    ### Eisenstein basis class for quasimodular forms of weight $k$ ###
    
    def __init__(self, weight=2):
        self.weight = weight
        self.dimension = len(Partitions((self.weight/2).floor(), max_part=3)) #Keeps track of dimension of QM_k

    def __repr__(self):
        return f"The Eisenstein basis of quasimodular forms of weight {self.weight}"
    
    
    def display(self):
        #Returns the dictionary {(i_1, i_2, ...):E_{i_1}E_{i_2}...}#
        basis = {}
        for i in Partitions((self.weight/2).floor(), max_part=3):
            basis_vector = 1
            for j in i:
                basis_vector *= eisenstein(2*j, self.dimension - 1)
            basis[tuple(2*k for k in i)] = truncate(basis_vector, self.dimension - 1)
        return basis
    
    
    def to_matrix(self):
        # Returns the square matrix a_{ij} = ith coefficient of the jth basis vector #
        m = []
        for basis_vector in self.display():
            m.append([i for i in self.display()[basis_vector]])
        return Matrix(m).transpose()

In [7]:
class Stationary_Descendent:
    ### The stationary descendent class, allows one to access individual intersection numbers ###
    ### and their generating polynomials ###
    
    def __init__(self, insertions=[0]):
        self.insertions = insertions
        self.weight = sum(self.insertions) + 2*len(self.insertions)
        
    def __repr__(self):
        return f"Stationary descendent class with insertions {self.insertions}, and weight {self.weight}" 
    
    def evaluate(self, d = 0):
        ### Returns the individual intersection number $<\tau_{k_1}\ldots\tau_{k_n}>^{\bullet E}_d$ ###
        return descendent(self.insertions, d)
        
    def expand(self, order = 0):
        ### Returns the generating function $<\tau_{k_1}\ldots\tau_{k_n}>$ up to the prescribed order ###
        
        if order == 0:
            return descendent(self.insertions, 0)
        
        euler = 1
        descendent_summation = 0
        
        for i in range(1, order + 1):
            euler *= (1 - q^i)
        
        euler = truncate(euler, order)
    
        for d in range(order + 1):
            descendent_summation += descendent(self.insertions, d)*q^d
            
        return truncate(euler*descendent_summation, order)
    
    def to_eisenstein_basis(self):
        ### Returns the stationary descendent in terms of the eisenstein basis ###
        
        target_vector = vector([i for i in self.expand(Eisenstein_Basis(self.weight).dimension - 1)]) #The descendent is uniquely determined by the first $\text{QM}_k$ coefficien
        
        matrix = Eisenstein_Basis(self.weight).to_matrix()
        
        solution_vector = matrix.solve_right(target_vector)
        
        solution_dict = {}
        
        counter = 0
        
        for i in Eisenstein_Basis(self.weight).display():
            solution_dict[i] = solution_vector[counter]
            counter += 1
        
        return solution_dict

### Example. 

Suppose we want to work with the stationary descendent $\left<\tau_2^2\right>$. We can use the .evaluate() method to compute individual Gromov-Witten invariants $\left<\tau_2^2\right>^{\bullet E}_d$. For example, if we wanted to compute $\left<\tau_2^2\right>^{\bullet E}_3$, we can do 

In [8]:
desc = Stationary_Descendent([2, 2])
desc.evaluate(d = 3)

166577809/11059200

Which tells us that $\left<\tau_2^2\right>^{\bullet E}_d = \frac{166577809}{11059200}$. If we want to expand $\left<\tau_2^2\right>$ up to order, say, 4, we could do

In [9]:
desc.expand(order = 4)

7495801/69120*q^4 + 248437/17280*q^3 + 15703/23040*q^2 + 127/69120*q + 49/33177600

Which tells us that

\begin{equation*}
\left<\tau_2^2\right> := (q)_\infty\sum_{d \geq 0}\left<\tau_2^2\right>^{\bullet E}_d =  \frac{49}{33177600} + \frac{127}{69120}q + \frac{15703}{23040}q^2 + \frac{248437}{17280}q^3 + \frac{7495801}{69120}q^4 + O(q^5)
\end{equation*}

Next, we'll write a class that will help us express quasimodular forms in tersm of the **descendent basis of weight $k$**, that is, in the basis

\begin{equation*}
\left\{ \left<\tau_{wt.=2}\right>^a\left<\tau_{wt.=4}\right>^b\left<\tau_{wt.=6}\right>^c \right\}_{2a + 4b + 6c = k}
\end{equation*}

There are only 8 such bases for any given weight $k$, corresponding to one of the following presentations of $\text{QM}$:

1. $\mathbb{Q}\left[  \left<\tau_0\right>, \left<\tau_0^2\right>, \left<\tau_0^3\right> \right]$
2. $\mathbb{Q}\left[  \left<\tau_0\right>, \left<\tau_0^2\right>, \left<\tau_1^2\right> \right]$
3. $\mathbb{Q}\left[  \left<\tau_0\right>, \left<\tau_0^2\right>, \left<\tau_2\tau_0\right> \right]$
4. $\mathbb{Q}\left[  \left<\tau_0\right>, \left<\tau_0^2\right>, \left<\tau_4\right> \right]$
5. $\mathbb{Q}\left[  \left<\tau_0\right>, \left<\tau_2\right>, \left<\tau_0^3\right> \right]$
6. $\mathbb{Q}\left[  \left<\tau_0\right>, \left<\tau_2\right>, \left<\tau_1^2\right> \right]$
7. $\mathbb{Q}\left[  \left<\tau_0\right>, \left<\tau_2\right>, \left<\tau_2\tau_0\right> \right]$
8. $\mathbb{Q}\left[  \left<\tau_0\right>, \left<\tau_2\right>, \left<\tau_4\right> \right]$

In [10]:
class Descendent_Basis:
    ### The descendent basis class of weight $k$. ###
    
    def __init__(self, weight = 2, typ = 1):
        self.weight = weight
        self.type = typ
        self.dimension = len(Partitions((self.weight/2).floor(), max_part=3))
        
        
    def __repr__(self):
        return f"The descendent basis in weight {self.weight} and type {self.type}"
    
    
    def display(self):
        
        types = {}
        
        types[1] = [[0], [0, 0], [0, 0, 0]]
        types[2] = [[0], [0, 0], [1, 1]]
        types[3] = [[0], [0, 0], [2, 0]]
        types[4] = [[0], [0, 0], [4]]
        types[5] = [[0], [2], [0, 0, 0]]
        types[6] = [[0], [2], [1, 1]]
        types[7] = [[0], [2], [2, 0]]
        types[8] = [[0], [2], [4]]
            
        basis = {}
            
        for i in Partitions((self.weight/2).floor(), max_part=3):
            basis_key = []
            for j in i:
                basis_key.append(tuple(types[self.type][j - 1]))
            basis_key = tuple(basis_key)
            basis_vector = 1
            for j in i:
                basis_vector *= Stationary_Descendent(types[self.type][j - 1]).expand(self.dimension - 1)
            basis[basis_key] = truncate(basis_vector, self.dimension - 1)
        
        return basis
    
    
    def to_matrix(self):
        # Returns the square matrix a_{ij} = ith coefficient of the jth basis vector #
        m = []
        for basis_vector in self.display():
            m.append([i for i in self.display()[basis_vector]])
        return Matrix(m).transpose()
    
    
    def expand(self, coef = [-1/24]):
        ### Takes a list a_1, \ldots, a_d of d rational numbers, where d is the dimension of QM_k          ###
        ### and returns the quasimodular form, in the                                                      ###
        ### descendent basis, whose first d coefficients are a_1, \ldots, a_d                              ###
        
        expansion = {}
        
        if len(coef) != Eisenstein_Basis(self.weight).dimension:
            return False
        
        M = self.to_matrix()
        X = M.solve_right(vector(coef))
        
        counter = 0
        for basis_vector in self.display():
            expansion[basis_vector] = X[counter]
            counter += 1
        
        return expansion

Suppose you wanted to see the descendent basis of weight 12 of type 2. You can use the .display() method.

In [11]:
basis = Descendent_Basis(weight = 12, typ = 2); basis

The descendent basis in weight 12 and type 2

Now suppose you want to express the discriminant modular form $\Delta = q\prod_{n \geq 1}(1 - q^n)^{24}$ in the above descendent basis. We can use the .expand() method. This method requires that you provide a list of rational numbers $[a_1, \ldots, a_{d}]$, where $d$ is the dimension of $\text{QM}_k$. The method returns the quasimodular form, expanded in the descendent basis, whose first $d$ coefficients are $a_1, \ldots, a_d$.

Since $\text{QM}_{12}$ has dimension 7, we need the first 7 coefficients of $\Delta$, which happen to be $[0, 1, -24, 252, -1472, 4830, -6048]$.

In [12]:
basis.expand([0, 1, -24, 252, -1472, 4830, -6048])

{((1, 1), (1, 1)): -97200,
 ((1, 1), (0, 0), (0,)): 155520,
 ((1, 1), (0,), (0,), (0,)): -362880,
 ((0, 0), (0, 0), (0, 0)): 13824,
 ((0, 0), (0, 0), (0,), (0,)): -20736,
 ((0, 0), (0,), (0,), (0,), (0,)): 331776,
 ((0,), (0,), (0,), (0,), (0,), (0,)): -324864}

The above output means 

\begin{equation*}
\Delta = -97200\left<\tau_1^2\right>^2 + 155520\left<\tau_1^2\right>\left<\tau_0^2\right>\left<\tau_0\right> - 362880\left<\tau_1^2\right>\left<\tau_0\right>^3 + 13824\left<\tau_0^2\right>^3 -20736\left<\tau_0^2\right>^2\left<\tau_0\right>^2 + 331776\left<\tau_0^2\right>\left<\tau_0\right>^4 - 324864\left<\tau_0\right>^6
\end{equation*}

# The Descendent_Matroid() Class

Now we'll make the Descendent_Matroid() class below.

In [13]:
class Descendent_Matroid:
    
    def __init__(self, weight = 2):
        self.weight = weight
        
    def __repr__(self):
        return f"The descendent matroid class of weight {self.weight}"
    
    def groundset(self, positive = False, nonpositive = False):
        # Returns the ground set of the descendent matroid of weight k #
        
        s = [] #Create the list that eventually holds the ground set
        
        #One pointed invariants --> partitions of weight-2 with at most 1 part
        #Two pointed invariants --> partitions of weight-4 with at most 2 parts
        #...
        #(weight/2) pointed invariants --> partitions of 0 with at most weight/2 parts
        
        if positive == False and nonpositive == False:
            for i in range(1, (self.weight/2).floor() + 1): #loop through "with at most __ parts"
                for p in Partitions(self.weight - 2*i, max_length = i).list():
                    l = list(p)
                    if len(p) != i: #Throw in zeroes so that the descendent has exactly the right amount of points
                        for j in range(1, i - len(p) + 1):
                            l.append(0)
                        s.append(l)
                    else:
                        s.append(l)
            return s
        
        if positive == True and nonpositive == False:
            for desc in self.groundset():
                if all([v > 0 for v in desc]):
                    s.append(desc)
            return s
        
        if nonpositive == True and positive == False:
            for desc in self.groundset():
                if all([v > 0 for v in desc]) == False:
                    s.append(desc)
            return s
    
    
    def matrix_repr(self, positive = False, nonpositive = False):
        # Returns the matrix A, whose columns correspond to the groundset, and  #
        # each column represents the descendent written in the Eisenstein basis # 
        # By default
        
        m = []
        
        if positive == False and nonpositive == False:
            for desc in self.groundset():
                column = []
                f = Stationary_Descendent(desc).to_eisenstein_basis()
                for basis_vector in f:
                    column.append(f[basis_vector])
                m.append(column)
        
            M = Matrix(m).transpose()
        
            return M
        
        if positive == True and nonpositive == False:
            for desc in self.groundset(positive = True):
                column = []
                f = Stationary_Descendent(desc).to_eisenstein_basis()
                for basis_vector in f:
                    column.append(f[basis_vector])
                m.append(column)
        
            M = Matrix(m).transpose()
        
            return M
        
        if nonpositive == True and positive == False:
            for desc in self.groundset(nonpositive = True):
                column = []
                f = Stationary_Descendent(desc).to_eisenstein_basis()
                for basis_vector in f:
                    column.append(f[basis_vector])
                m.append(column)
        
            M = Matrix(m).transpose()
        
            return M
        
        
    
    def matroid_repr(self, positive = False, nonpositive = False):
        # Returns the matroid class ##
        
        if positive == False and nonpositive == False:
            mat = Matroid(self.matrix_repr())
            return mat
        
        if positive == True and nonpositive == False:
            mat = Matroid(self.matrix_repr(positive = True))
            return mat
        
        if nonpositive == True and positive == False:
            mat = Matroid(self.matrix_repr(nonpositive = True))
            return mat
    
    def bases(self, positive = False, nonpositive = False):
        # Returns the bases of the descendent matroid #
        set_of_bases = []
        
        if positive == False and nonpositive == False:
            for b in self.matroid_repr().bases():
                basis = []
                for index in b:
                    basis.append(self.groundset()[index])
                set_of_bases.append(basis)
            return iter(set_of_bases)
        
        if positive == True and nonpositive == False:
            for b in self.matroid_repr(positive = True).bases():
                basis = []
                for index in b:
                    basis.append(self.groundset(positive = True)[index])
                set_of_bases.append(basis)
            return iter(set_of_bases)
        
        if nonpositive == True and positive == False:
            for b in self.matroid_repr(nonpositive = True).bases():
                basis = []
                for index in b:
                    basis.append(self.groundset(nonpositive = True)[index])
                set_of_bases.append(basis)
            return iter(set_of_bases)

We can use the Descendent_Matroid() class to make computations with the descendent matroid $\mathcal{M}_k$.
First, let's instantiate $\mathcal{M}_k$ for $k = 2, 4, 6, 8, 10, 12$:

In [14]:
m2 = Descendent_Matroid(2)
m4 = Descendent_Matroid(4)
m6 = Descendent_Matroid(6)
m8 = Descendent_Matroid(8)
m10 = Descendent_Matroid(10)
m12 = Descendent_Matroid(12)

Let's compute the matrix representation of $\mathcal{M}_k$ for $k = 2, 4, 6, 8$:

In [15]:
print(m2.matrix_repr())
print("")
print(m4.matrix_repr())
print("")
print(m6.matrix_repr())
print("")
print(m8.matrix_repr())

[1]

[1/12  5/6]
[ 1/2   -1]

[1/360 7/120 7/180  7/12]
[ 1/12   1/4   2/3 -15/2]
[  1/6  -3/2  -8/3     3]

[  1/360    1/36  13/180    1/12   -7/15   7/180   -35/3]
[19/2016 115/504   25/63  73/112   85/24    25/9  325/12]
[   1/24    -1/3    -2/3    -3/4      -6   -38/3      75]
[   1/24    -5/6    -8/3   -15/4    15/2    40/3     -15]


If you want to know which column corresponds to which stationary descendent, you can use the .groundset() method:

In [16]:
m8.groundset()

[[6], [4, 0], [3, 1], [2, 2], [2, 0, 0], [1, 1, 0], [0, 0, 0, 0]]

The above output means that, when you look at the $(4 \times 7)$ matrix corresponding to $\mathcal{M}_8$, the first column corresponds to $\left<\tau_6\right>$, the second column corresponds to $\left<\tau_4\tau_0\right>$, etc.

The method .matroid_repr() allows us to access $\mathcal{M}_k$ using Sage's Matroids library:

In [17]:
mat8 = m8.matroid_repr(); mat8

Linear matroid of rank 4 on 7 elements represented over the Rational Field

The Sage library for the Matroids class has many build in methods.

In [18]:
mat8.bases_count()

34

In [19]:
mat8.tutte_polynomial()

x^4 + 3*x^3 + y^3 + 6*x^2 + x*y + 4*y^2 + 9*x + 9*y

We can also access the restricted matroid $\mathcal{M}_k | S^k$ by setting the `positive' parameter in either the .matrix_repr() method or the .matroid_repr() method to True.

In [20]:
m12.matrix_repr(positive = True)

[  293/37065600       35/28512      133/38016    1421/237600        23/3168       301/4320       539/4320       539/2880     7399/10800]
[      23/47520      703/23760     1207/15840      1063/7920          16/99       -379/720       -859/720         -37/16       -1043/45]
[        1/2160        -11/216        -19/144        -83/360          -5/18         -61/24        -283/72          -41/8         812/45]
[10181/10378368       475/3564     4729/12672      1775/2772      7405/9504        725/108         325/28       7591/448        1750/27]
[       19/4032         -11/63         -29/64         -17/21         -47/48            -27        -907/21     -12241/224         -796/3]
[         1/288           -7/9        -203/96          -11/3          -35/8           85/3          121/3         765/16         2368/3]
[         1/720           -4/9         -21/16          -12/5         -35/12          140/3             84          945/8        -2240/3]

In [21]:
mat12 = m12.matroid_repr(positive = True); mat12

Linear matroid of rank 7 on 9 elements represented over the Rational Field

In [22]:
mat12.bases_count() == binomial(9, 7)

True